In [ ]:
import os
import numpy as np
import pandas as pd  
import matplotlib.pyplot as plt

import bigfish
import bigfish.stack as stack
import bigfish.plot as plot
import bigfish.multistack as multistack
import bigfish.segmentation as segmentation 
import bigfish.detection as detection  

import tifffile as tf
import re 

import threading
import datetime as dt
import gc

from roifile import ImagejRoi
from imaris_ims_file_reader.ims import ims

In [2]:
path = ["D:/round1/RNAscope_time/Data_ROI"]
out_path = "D:/round1/RNAscope_time/result"

In [3]:
## multiple thread
max_connections = 1
pool_sema = threading.BoundedSemaphore(max_connections)

# Function: detect spots

In [4]:
def detect_spot(image, z_radius, xy_radius):
    spots, threshold = detection.detect_spots(
        images=image,
        return_threshold=True,
        voxel_size=(1000, 191, 191),
        spot_radius=(z_radius, xy_radius, xy_radius))#可调
    
    spots_post_decomposition, dense_regions, reference_spot = detection.decompose_dense(
        image=image, 
        spots=spots, 
        voxel_size=(1000, 191, 191), 
        spot_radius=(z_radius, xy_radius, xy_radius), 
        alpha=0.7,  # alpha impacts the number of spots per candidate region
        beta=1,  # beta impacts the number of candidate regions to decompose
        gamma=5)  # gamma the filtering step to denoise the image
    
    spots_post_clustering, clusters = detection.detect_clusters(
        spots=spots_post_decomposition, 
        voxel_size=(1000, 191, 191), 
        radius=1000, 
        nb_min_spots=10)
    
    print("detected spots before decomposition")
    print("\r shape: {0}".format(spots.shape))
    print("\r dtype: {0}".format(spots.dtype), "\n")
    print("detected spots after decomposition")
    print("\r shape: {0}".format(spots_post_decomposition.shape))
    print("\r dtype: {0}".format(spots_post_decomposition.dtype))
    print("detected spots after clustering")
    print("\r shape: {0}".format(spots_post_clustering.shape))
    print("\r dtype: {0}".format(spots_post_clustering.dtype), "\n")
    print("detected clusters")
    print("\r shape: {0}".format(clusters.shape))
    print("\r dtype: {0}".format(clusters.dtype))

    return spots_post_decomposition, clusters


In [5]:
def plot_spot(image, spots_post_decomposition, clusters):
    image_mip = stack.maximum_projection(image)
    plot.plot_detection(image_mip, 
                        spots=[spots_post_decomposition, clusters[:, :3]], 
                        shape=["circle", "square"], 
                        radius=[3, 6], 
                        color=["red", "blue"],
                        linewidth=[1, 2], 
                        fill=[False, True], 
                        contrast=True)
    return

In [6]:
def extract_roifile(img_name, path):
    roi_name1 = re.sub(pattern="\.ims", repl=".zip", string=img_name)
    roi_name2 = re.sub(pattern="\.ims", repl=".roi.zip", string=img_name)

    try:
        roi = ImagejRoi.fromfile(path + "/" + roi_name1)
        print("found roi of img: %s" % (img_name))
        return roi_name1
    except:
        pass

    try:
        roi = ImagejRoi.fromfile(path + "/" + roi_name2)
        print("found roi of img: %s" % (img_name))
        return roi_name2
    except:
        pass

    return 0   

In [7]:
def extract_result(img_name, path, out_dir):
    pool_sema.acquire()

    roi_name = extract_roifile(img_name, path)

    if (roi_name == 0):
        print("no roi file found: %s" % (img_name))
        pool_sema.release()
        return 

    print('current processing image: %s/%s\n' % (path, img_name))

    genotype = re.sub(pattern='_.*', repl='', string=img_name)

    ZT = re.sub(pattern='.*ZT', repl='ZT', string=img_name)
    ZT = re.sub(pattern='_.*', repl='', string=ZT)

    number = re.sub(pattern='.*LN._', repl='', string=img_name)
    number = re.sub(pattern='\..*', repl='', string=number)
    number = re.sub(pattern='_.*', repl='', string=number)

    side = re.sub(pattern='.*LN', repl='LN', string=img_name)
    side = re.sub(pattern='_.*', repl='', string=side)

    try:
        img = ims(path + "/" + img_name)
    except:
        print("unreadable image: %s/%s\n, skiped" %(path, img_name))
        pool_sema.release()
        return
 
    roi = ImagejRoi.fromfile(path + "/" + roi_name)

    try: 
        spots_c1, clusters_c1 = detect_spot(image=img[0, 0], xy_radius=80, z_radius=400)
    except:
        print("unprocessable image for c1 channel:  %s/%s\n, skiped" %(path, img_name))
        pool_sema.release()
        return
    
    try: 
        spots_c2, clusters_c2 = detect_spot(image=img[0, 1], xy_radius=80, z_radius=400)
    except:
        print("unprocessable image for c2 channel:  %s/%s\n, skiped" %(path, img_name))
        pool_sema.release()
        return 

    cell_spot_sum = pd.DataFrame(data=None, columns = ['Genotype', 'ZT', 'number', 'side', ' channel', 'cell', 'spot_num']) 
    cell_spot = pd.DataFrame(data=None, columns = ['Genotype', 'ZT', 'number', 'side', 'channel', 'cell', 'x', 'y', 'z'])

    for r in range(len(roi)):
        
        roi_cell = roi[r].name

        if (re.match(pattern = ".*_chr$", string=roi_cell) != None):
            spot_z_c1 =  spots_c1[np.where((spots_c1[:, 0] >= roi[r].z_position-3) & (spots_c1[:, 0] <= roi[r].z_position+1))]
            spot_z_c2 =  spots_c2[np.where((spots_c2[:, 0] >= roi[r].z_position-3) & (spots_c2[:, 0] <= roi[r].z_position+1))]
        elif (re.match(pattern = "lLNv.*", string=roi_cell) != None):
            spot_z_c1 =  spots_c1[np.where((spots_c1[:, 0] >= roi[r].z_position-7) & (spots_c1[:, 0] <= roi[r].z_position+5))]
            spot_z_c2 =  spots_c2[np.where((spots_c2[:, 0] >= roi[r].z_position-7) & (spots_c2[:, 0] <= roi[r].z_position+5))]
        else:
            spot_z_c1 =  spots_c1[np.where((spots_c1[:, 0] >= roi[r].z_position-5) & (spots_c1[:, 0] <= roi[r].z_position+3))]
            spot_z_c2 =  spots_c2[np.where((spots_c2[:, 0] >= roi[r].z_position-5) & (spots_c2[:, 0] <= roi[r].z_position+3))]

        # print(len(spot_z))
        
        cell_spot_sum.loc[len(cell_spot_sum.index)] = [genotype, ZT, number, side, 'C1', roi_cell, 0] 
  
        for i in range(spot_z_c1.shape[0]): 
            border_z_c1 = roi[r].integer_coordinates[np.where(roi[r].integer_coordinates[:, 1] + roi[r].top == spot_z_c1[i, 1])]
            
            if (border_z_c1.shape[0] == 0) : 
                # print("not in: spot %s"%(i))  
                continue

            if ((np.min(border_z_c1[:, 0])+roi[r].left <= spot_z_c1[i, 2]) & (np.max(border_z_c1[:, 0])+roi[r].left >= spot_z_c1[i, 2] )):
                # print("sp  ot: %s(x: %s  y: %s  z: %s)"%(i, spot_z[i, 1], spot_z[i, 2], spot_z[i, 0]))
                cell_spot_sum.iloc[len(cell_spot_sum.index)-1, 6] = cell_spot_sum.iloc[len(cell_spot_sum.index)-1, 6] + 1
                cell_spot.loc[len(cell_spot.index)] = [genotype, ZT, number, side, 'C1', roi_cell, spot_z_c1[i, 1], spot_z_c1[i, 2], spot_z_c1[i, 0]]

        cell_spot_sum.loc[len(cell_spot_sum.index)] = [genotype, ZT, number, side, 'C2', roi_cell, 0]

        for i in range(spot_z_c2.shape[0]):
            border_z_c2 = roi[r].integer_coordinates[np.where(roi[r].integer_coordinates[:, 1] + roi[r].top == spot_z_c2[i, 1])]
             
            if (border_z_c2.shape[0] == 0) :
                # print("not in: spot %s"%(i)) 
                continue

            if ((np.min(border_z_c2[:, 0])+roi[r].left <= spot_z_c2[i, 2]) & (np.max(border_z_c2[:, 0])+roi[r].left >= spot_z_c2[i, 2] )):
                # print("spot: %s(x: %s  y: %s  z: %s)"%(i, spot_z[i, 1], spot_z[i, 2], spot_z[i, 0]))
                cell_spot_sum.iloc[len(cell_spot_sum.index)-1, 6] = cell_spot_sum.iloc[len(cell_spot_sum.index)-1, 6] + 1
                cell_spot.loc[len(cell_spot.index)] = [genotype, ZT, number, side, 'C2', roi_cell, spot_z_c2[i, 1], spot_z_c2[i, 2], spot_z_c2[i, 0]]    

    cell_spot_sum.to_csv("%s/%s.txt" % (out_dir, genotype), sep = '\t', index = False, header= False, mode = 'a')
    cell_spot.to_csv("%s/%s_spot.txt" % (out_dir, genotype), sep = '\t', index = False, header= False, mode = 'a')

    pool_sema.release()

    return 

In [8]:
def main():
    thread_list = []
    for i in range(len(path)):
        img_list_all = os.listdir(path[i])
        img_list = list(filter(lambda x: re.match('.*\.ims', x) != None, img_list_all))
        # img_list1 = list(map(lambda x : re.sub('\..*', '.tif', x), img_list))
        # if not os.path.exists(path[i] + "/Split/Spot/"):
        #     os.makedirs(path[i] + "/Split/Spot/")

        # img_list = [img_list[0]]

        for j in range(len(img_list)):
            # t = threading.Thread(target=test_thread, args = [j, img_list[j]])
            t = threading.Thread(target = extract_result, args = [img_list[j], path[i], out_path])
            thread_list.append(t)

    for t in thread_list:
        t.start() 

    for t in thread_list:
        t.join()

In [ ]:
if __name__ == "__main__":
    main()

found roi of img: 12758_405DAPI_488timCDS_561PDF_647intron_ZT14_LN2_1_2024-10-04_13.43.40.ims
current processing image: D:/round1/RNAscope_time/Data_ROI/12758_405DAPI_488timCDS_561PDF_647intron_ZT14_LN2_1_2024-10-04_13.43.40.ims

Opening readonly file: D:/round1/RNAscope_time/Data_ROI/12758_405DAPI_488timCDS_561PDF_647intron_ZT14_LN2_1_2024-10-04_13.43.40.ims 

unprocessable image for c1 channel:  D:/round1/RNAscope_time/Data_ROI/12758_405DAPI_488timCDS_561PDF_647intron_ZT14_LN2_1_2024-10-04_13.43.40.ims
, skiped
Closing file: D:/round1/RNAscope_time/Data_ROI/12758_405DAPI_488timCDS_561PDF_647intron_ZT14_LN2_1_2024-10-04_13.43.40.ims 


In [ ]:
roi[1].plot()

In [ ]:
ap = roi1[0].plot()

In [ ]:
a = plt.figure()
plt.scatter(spot_z[:, 1], spot_z[:, 2])
plt.plot(roi1[0].integer_coordinates[:,0]+roi1[0].left, roi1[0].integer_coordinates[:,1]+roi1[0].top )
# ax = plt.gca()
# ax.set_aspect(1)

In [ ]:

for i in range(spot_z.shape[0]):
    border_z = roi1[0].integer_coordinates[np.where(roi1[0].integer_coordinates[:, 1] + roi1[0].top == spot_z[i, 2])]
    
    if (border_z.shape[0] == 0) :
        # print("not in: spot %s"%(i))
        continue

    if ((np.min(border_z[:, 0])+roi1[0].left <= spot_z[i, 1]) & (np.max(border_z[:, 0])+roi1[0].left >= spot_z[i, 1] )):
        print("spot: %s(x: %s  y: %s  z: %s)"%(i, spot_z[i, 1], spot_z[i, 2], spot_z[i, 0]))

In [ ]:
np.min(roi1[0].integer_coordinates[:, 1])

In [ ]:
roi1[0]

In [ ]:
a = plt.plot(roi1[0].integer_coordinates[:,0]+roi1[0].left, roi1[0].integer_coordinates[:,1]+roi1[0].top )
ax = plt.gca()
ax.set_aspect(1)
# ax.set_ylim((80, -5))

In [ ]:
roi1[0]

# Extract cell level result

### segmentation (grey)

In [53]:
sample = "12758_ZT14_LN2_1"

In [11]:
ovules = tf.imread("gray_filter_ovules.tif")
LRP = tf.imread("gray_filter_LRP.tif") 
              
seg = pd.read_excel("Segmentation_info.xlsx") 

In [13]:
seg['spots'] = 0
seg.head()

In [ ]:
for i in range(seg.shape[0]):
    print(seg.iloc[i, 0])
    if (re.match("ovules", seg.iloc[i, 1], re.IGNORECASE)):
        cell_loc_3 = np.where(ovules == seg.iloc[i, 2])
    elif (re.match("LRP", seg.iloc[i, 1], re.IGNORECASE)):
        cell_loc_3 = np.where(LRP == seg.iloc[i, 2]) 
    cell_loc = np.concatenate((cell_loc_3[0][:, np.newaxis],
                                cell_loc_3[1][:, np.newaxis],
                                cell_loc_3[2][:, np.newaxis]), axis = 1)
    cell_spots = np.where((spots_post_decomposition[:, None, :] == cell_loc).all(-1))[1]
    seg.iloc[i, 3] = cell_spots.shape[0]
                                

In [39]:
seg['repeat'] = seg.groupby('number')['number'].transform('count')
seg['spots'] = seg['spots'] / seg['repeat']
seg.head()

In [52]:
seg.to_excel("Segmentation_info.xlsx", index = False, sheet_name = "12758_ZT14_LN2_1")

# Old


In [ ]:
def extract_result_old(roi_name, path, out_dir):
    pool_sema.acquire()

    img_name = re.sub(pattern="\.zip", repl=".tif", string=roi_name)
    img_name = re.sub(pattern='\.roi', repl='', string=img_name)
    img_name_c2 = re.sub(pattern='C1_', repl='C2_', string=img_name)

    print('current processing image: %s/C1/%s\n' % (path, img_name))
    
    genotype = re.sub(pattern='C1_', repl='', string=img_name)
    genotype = re.sub(pattern='_.*', repl='', string=genotype)

    ZT = re.sub(pattern='.*Dil_', repl='', string=img_name)
    ZT = re.sub(pattern='_.*', repl='', string=ZT)

    number = re.sub(pattern='.*LN._', repl='', string=img_name)
    number = re.sub(pattern='\..*', repl='', string=number)
    number = re.sub(pattern='_.*', repl='', string=number)

    try:
        img = tf.imread(path + "/C1/" + img_name)
    except:
        print("unreadable image: %s/C1/%s\n, skiped" %(path, img_name))
        return
    
    try:
        img_c2 = tf.imread(path + "/C2/" + img_name_c2)
    except:
        print("unreadable image: %s/C2/%s\n, skiped" %(path, img_name_c2))
        return

    roi = ImagejRoi.fromfile(path + "/C1/" + roi_name)

    try: 
        spots, clusters = detect_spot(image=img, xy_radius=80, z_radius=400)
    except:
        print("unprocessable image:  %s/C1/%s\n, skiped" %(path, img_name))
        return
    
    try: 
        spots_c2, clusters_c2 = detect_spot(image=img_c2, xy_radius=80, z_radius=400)
    except:
        print("unprocessable image:  %s/C2/%s\n, skiped" %(path, img_name_c2))
        return 

    cell_spot_sum = pd.DataFrame(data=None, columns = ['Genotype', 'ZT', 'number', ' channel', 'cell', 'spot_num']) 
    cell_spot = pd.DataFrame(data=None, columns = ['Genotype', 'ZT', 'number', 'channel', 'cell', 'x', 'y', 'z'])

    for r in range(len(roi)):
        
        roi_cell = roi[r].name

        if (re.match(pattern = ".*_chr$", string=roi_cell) != None):
            spot_z =  spots[np.where((spots[:, 0] >= roi[r].z_position-2) & (spots[:, 0] <= roi[r].z_position))]
            spot_z_c2 =  spots_c2[np.where((spots_c2[:, 0] >= roi[r].z_position-2) & (spots_c2[:, 0] <= roi[r].z_position))]
        elif (re.match(pattern = "lLNv.*", string=roi_cell) != None):
            spot_z =  spots[np.where((spots[:, 0] >= roi[r].z_position-4) & (spots[:, 0] <= roi[r].z_position+2))]
            spot_z_c2 =  spots_c2[np.where((spots_c2[:, 0] >= roi[r].z_position-4) & (spots_c2[:, 0] <= roi[r].z_position+2))]
        else:
            spot_z =  spots[np.where((spots[:, 0] >= roi[r].z_position-3) & (spots[:, 0] <= roi[r].z_position+1))]
            spot_z_c2 =  spots_c2[np.where((spots_c2[:, 0] >= roi[r].z_position-3) & (spots_c2[:, 0] <= roi[r].z_position+1))]

        # print(len(spot_z))
        
        cell_spot_sum.loc[len(cell_spot_sum.index)] = [genotype, ZT, number, '1', roi_cell, 0] 
  
        for i in range(spot_z.shape[0]): 
            border_z = roi[r].integer_coordinates[np.where(roi[r].integer_coordinates[:, 1] + roi[r].top == spot_z[i, 1])]
            
            if (border_z.shape[0] == 0) : 
                # print("not in: spot %s"%(i))  
                continue

            if ((np.min(border_z[:, 0])+roi[r].left <= spot_z[i, 2]) & (np.max(border_z[:, 0])+roi[r].left >= spot_z[i, 2] )):
                # print("sp  ot: %s(x: %s  y: %s  z: %s)"%(i, spot_z[i, 1], spot_z[i, 2], spot_z[i, 0]))
                cell_spot_sum.iloc[len(cell_spot_sum.index)-1, 5] = cell_spot_sum.iloc[len(cell_spot_sum.index)-1, 5] + 1
                cell_spot.loc[len(cell_spot.index)] = [genotype, ZT, number, '1', roi_cell, spot_z[i, 1], spot_z[i, 2], spot_z[i, 0]]

        cell_spot_sum.loc[len(cell_spot_sum.index)] = [genotype, ZT, number, '2', roi_cell, 0]

        for i in range(spot_z_c2.shape[0]):
            border_z_c2 = roi[r].integer_coordinates[np.where(roi[r].integer_coordinates[:, 1] + roi[r].top == spot_z_c2[i, 1])]
             
            if (border_z_c2.shape[0] == 0) :
                # print("not in: spot %s"%(i)) 
                continue

            if ((np.min(border_z_c2[:, 0])+roi[r].left <= spot_z_c2[i, 2]) & (np.max(border_z_c2[:, 0])+roi[r].left >= spot_z_c2[i, 2] )):
                # print("spot: %s(x: %s  y: %s  z: %s)"%(i, spot_z[i, 1], spot_z[i, 2], spot_z[i, 0]))
                cell_spot_sum.iloc[len(cell_spot_sum.index)-1, 5] = cell_spot_sum.iloc[len(cell_spot_sum.index)-1, 5] + 1
                cell_spot.loc[len(cell_spot.index)] = [genotype, ZT, number, '2', roi_cell, spot_z[i, 1], spot_z[i, 2], spot_z[i, 0]]    

    cell_spot_sum.to_csv("%s/%s.txt" % (out_dir, genotype), sep = '\t', index = False, header= False, mode = 'a')
    cell_spot.to_csv("%s/%s_spot.txt" % (out_dir, genotype), sep = '\t', index = False, header= False, mode = 'a')

    pool_sema.release()

    return 
    
    